In [48]:
import os
import pandas as pd
from pandas import DataFrame
from datetime import datetime as dt
import matplotlib.pyplot as plt

# Imports the method used for connecting to DBs
from sqlalchemy import create_engine, inspect

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Boolean 

from sqlalchemy.orm import Session

In [49]:
engine = create_engine("sqlite:///../Data/megawatt.sqlite")

In [50]:
newdf = DataFrame(engine.execute("SELECT * FROM demand where date >'2016-01-01'and date < '2018-12-01' and name='ISNE'").fetchall())
newdf = newdf.rename(columns={
    newdf.columns[0]: 'name',
    newdf.columns[1]: 'datetime',
    newdf.columns[2]: 'value',
    newdf.columns[3] : 'date',
    newdf.columns[4] : 'hour'
    })

newdf['datetime'] = pd.to_datetime(newdf['datetime'])
# newdf.sort_values('datetime',ascending=True)
newdf.head()


,name,datetime,value,date,hour
0,ISNE,2018-10-17 02:00:00,13130.0,2018-10-17,2
1,ISNE,2018-10-17 01:00:00,14055.0,2018-10-17,1
2,ISNE,2018-10-17 00:00:00,14683.0,2018-10-17,0
3,ISNE,2018-10-16 23:00:00,14698.0,2018-10-16,23
4,ISNE,2018-10-16 22:00:00,13792.0,2018-10-16,22


In [51]:
weather_file = os.path.join('../Data','2016-2018_MA_weather_data.csv')

In [52]:
raw_listings_df = pd.read_csv(weather_file, encoding="ISO-8859-1")

In [53]:
raw_listings_df['datetime'] = pd.to_datetime(raw_listings_df['datetime'])
raw_listings_df.head()

,Unnamed: 0,usaf,wban,windir,windspeed,skycond,tempc,dewpoint,tempf,datetime
0,0,725090,14739,270,57,99999,0.56,31.604,33.008,2016-01-01 00:00:00
1,1,725090,14739,260,51,22000,0.50,31.604,32.900,2016-01-01 01:00:00
2,2,725090,14739,260,62,1433,0.50,31.496,32.900,2016-01-01 02:00:00
3,3,725090,14739,260,46,1433,0.44,31.496,32.792,2016-01-01 03:00:00
4,5,725090,14739,260,57,1311,0.44,31.496,32.792,2016-01-01 04:00:00


In [54]:
# raw_listings_df.dtypes

In [55]:
whatdf = pd.merge(newdf, raw_listings_df, on='datetime',how='inner')
whatdf.head()

,name,datetime,value,date,hour,Unnamed: 0,usaf,wban,windir,windspeed,skycond,tempc,dewpoint,tempf
0,ISNE,2018-10-16 05:00:00,11323.0,2018-10-16,5,11263,725090,14739,999,9999,99999,99.99,211.982,211.982
1,ISNE,2018-10-15 21:00:00,14670.0,2018-10-15,21,11261,725090,14739,210,67,366,1.83,34.808,35.294
2,ISNE,2018-10-15 20:00:00,14362.0,2018-10-15,20,11259,725090,14739,200,77,457,1.78,34.592,35.204
3,ISNE,2018-10-15 19:00:00,14350.0,2018-10-15,19,11258,725090,14739,190,82,884,1.72,34.394,35.096
4,ISNE,2018-10-15 18:00:00,14555.0,2018-10-15,18,11254,725090,14739,180,67,853,1.60,34.340,34.880


In [56]:
whatdf.to_csv('out.csv')